# EMG信号可视化入门

本教程将教你如何可视化EMG信号的各种特性。

## 学习内容
1. 时域波形绘制
2. 频域分析（频谱图）
3. 时频分析（时频谱图）
4. 多通道信号对比
5. 统计特性可视化

In [ ]:
# ===== 导入必要的Python库 =====

# 系统库
import sys  # 用于操作Python环境
from pathlib import Path  # 用于处理文件路径

# 科学计算和数据处理库
import numpy as np  # 数值计算：数组、矩阵、数学函数
import matplotlib.pyplot as plt  # 绘图库：创建各种图表
from scipy import signal  # 信号处理库：频谱分析、滤波等

# ===== 添加项目路径 =====
project_root = Path.cwd().parent  # 获取项目根目录
sys.path.insert(0, str(project_root))  # 添加到Python搜索路径

# --- Jupyter/IPython 兼容性修复 ---
# IPython 往往会先导入标准库的 `code` 模块，导致 `code.weekXX_*` 不是包。
# 这里把项目的 `code/` 目录挂到 `code.__path__`，让子模块可被正常导入。
import code as _code
if not hasattr(_code, '__path__'):
    _code.__path__ = [str(project_root / 'code')]

# ===== 配置matplotlib中文显示 =====
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Segoe UI Symbol', 'Arial Unicode MS', 'DejaVu Sans']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.rcParams['figure.dpi'] = 100  # 设置图表清晰度（DPI：每英寸点数）

print("✓ 库导入成功！")

## 1. 生成模拟EMG信号

首先生成一个包含静息期和收缩期的EMG信号。

In [ ]:
# ===== 生成模拟EMG信号 =====

# --- 第一步：设置信号参数 ---
fs = 1000  # 采样率：每秒1000个采样点（1000 Hz）
duration = 5  # 信号时长：5秒
t = np.linspace(0, duration, int(duration * fs))
# linspace生成均匀分布的时间点：从0到5秒，共5000个点

# --- 第二步：生成基础噪声（静息状态） ---
signal_emg = np.random.normal(0, 0.02, len(t))
# normal(均值, 标准差, 数量)
# 生成高斯噪声，模拟肌肉完全放松时的微弱电活动

# --- 第三步：定义肌肉收缩期（1-4秒） ---
contraction_start = int(1 * fs)  # 1秒 = 第1000个采样点
contraction_end = int(4 * fs)    # 4秒 = 第4000个采样点

# --- 第四步：在收缩期添加多个频率成分 ---
# EMG信号是多个运动单元放电的叠加
for freq in range(60, 150, 15):  # 频率：60, 75, 90, 105, 120, 135 Hz
    amplitude = np.random.uniform(0.2, 0.4)  # 随机振幅：0.2-0.4 mV
    # 每个运动单元的振幅略有不同
    
    signal_emg[contraction_start:contraction_end] += amplitude * np.sin(
        2 * np.pi * freq * t[contraction_start:contraction_end]
    )
    # 正弦波公式：A * sin(2πft)
    # A: 振幅，f: 频率，t: 时间

# --- 第五步：添加随机噪声（模拟信号干扰） ---
signal_emg[contraction_start:contraction_end] += np.random.normal(
    0, 0.15, contraction_end - contraction_start
)
# 收缩期的噪声更大（标准差0.15），模拟肌肉收缩时的复杂性

# --- 第六步：打印信号信息 ---
print(f"✓ 信号生成完成")
print(f"  采样点数: {len(signal_emg)}")
print(f"  采样率: {fs} Hz")
print(f"  时长: {duration} 秒")
print(f"  收缩期: {contraction_start/fs:.1f}-{contraction_end/fs:.1f} 秒")

## 2. 时域波形可视化

最基本的可视化：显示信号随时间的变化。

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 1. 完整信号
axes[0].plot(t, signal_emg, linewidth=0.5, color='blue')
axes[0].axvspan(0, 1, alpha=0.2, color='green', label='静息期')
axes[0].axvspan(1, 4, alpha=0.2, color='red', label='收缩期')
axes[0].axvspan(4, 5, alpha=0.2, color='green', label='放松期')
axes[0].set_ylabel('幅度 (mV)', fontsize=12)
axes[0].set_title('完整EMG信号：静息 → 收缩 → 放松', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. 静息期放大（前0.5秒）
rest_end = int(0.5 * fs)
axes[1].plot(t[:rest_end], signal_emg[:rest_end], linewidth=0.8, color='green')
axes[1].set_ylabel('幅度 (mV)', fontsize=12)
axes[1].set_title('静息期放大：低幅度基线噪声', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(-0.15, 0.15)

# 3. 收缩期放大（1.5-2秒）
contract_start = int(1.5 * fs)
contract_end = int(2.0 * fs)
axes[2].plot(
    t[contract_start:contract_end],
    signal_emg[contract_start:contract_end],
    linewidth=0.8,
    color='red'
)
axes[2].set_xlabel('时间 (秒)', fontsize=12)
axes[2].set_ylabel('幅度 (mV)', fontsize=12)
axes[2].set_title('收缩期放大：高幅度随机信号', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 时域波形绘制完成")

## 3. 频域分析（功率谱）

使用傅里叶变换分析信号的频率成分。

In [ ]:
# 分别分析静息期和收缩期的频谱
rest_signal = signal_emg[:int(1*fs)]
contract_signal = signal_emg[int(1*fs):int(4*fs)]

# 计算功率谱密度
freqs_rest, psd_rest = signal.welch(rest_signal, fs=fs, nperseg=256)
freqs_contract, psd_contract = signal.welch(contract_signal, fs=fs, nperseg=256)

# 绘制频谱图
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 静息期频谱
axes[0].semilogy(freqs_rest, psd_rest, linewidth=2, color='green')
axes[0].set_ylabel('功率谱密度', fontsize=12)
axes[0].set_title('静息期频谱：主要是低频噪声', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(0, 250)

# 收缩期频谱
axes[1].semilogy(freqs_contract, psd_contract, linewidth=2, color='red')
axes[1].set_xlabel('频率 (Hz)', fontsize=12)
axes[1].set_ylabel('功率谱密度', fontsize=12)
axes[1].set_title('收缩期频谱：60-150 Hz范围内有明显峰值', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, 250)

# 标注主要频率范围
axes[1].axvspan(60, 150, alpha=0.2, color='yellow', label='主要频率范围')
axes[1].legend()

plt.tight_layout()
plt.show()

print("✓ 频域分析完成")

## 4. 时频分析（时频谱图）

使用短时傅里叶变换(STFT)观察频率随时间的变化。

In [ ]:
# 计算时频谱图
f, t_spec, Sxx = signal.spectrogram(signal_emg, fs=fs, nperseg=256, noverlap=200)

# 绘制时频谱图
plt.figure(figsize=(14, 6))
plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
plt.ylabel('频率 (Hz)', fontsize=12)
plt.xlabel('时间 (秒)', fontsize=12)
plt.title('EMG信号时频谱图', fontsize=14, fontweight='bold')
plt.colorbar(label='功率 (dB)')
plt.ylim(0, 250)

# 标注不同阶段
plt.axvline(1, color='white', linestyle='--', linewidth=2, label='收缩开始')
plt.axvline(4, color='white', linestyle='--', linewidth=2, label='收缩结束')
plt.legend()

plt.tight_layout()
plt.show()

print("✓ 时频分析完成")
print("  可以看到收缩期（1-4秒）频率能量明显增强")

## 5. 多通道信号可视化

模拟4通道EMG信号并进行对比显示。

In [ ]:
# 生成4通道信号（每个通道略有不同）
n_channels = 4
signals_multi = []

for ch in range(n_channels):
    # 基础噪声
    sig = np.random.normal(0, 0.02, len(t))
    
    # 收缩期，每个通道的强度略有不同
    intensity = np.random.uniform(0.8, 1.2)
    for freq in range(60, 150, 15):
        amplitude = np.random.uniform(0.2, 0.4) * intensity
        sig[contraction_start:contraction_end] += amplitude * np.sin(
            2 * np.pi * freq * t[contraction_start:contraction_end] + np.random.uniform(0, 2*np.pi)
        )
    
    sig[contraction_start:contraction_end] += np.random.normal(
        0, 0.15 * intensity, contraction_end - contraction_start
    )
    
    signals_multi.append(sig)

# 绘制多通道信号
fig, axes = plt.subplots(n_channels, 1, figsize=(14, 10))
fig.suptitle('4通道EMG信号', fontsize=16, fontweight='bold')

for ch in range(n_channels):
    axes[ch].plot(t, signals_multi[ch], linewidth=0.5, color=f'C{ch}')
    axes[ch].set_ylabel(f'通道{ch+1}\n(mV)', fontsize=10)
    axes[ch].grid(True, alpha=0.3)
    axes[ch].set_ylim(-2, 2)
    
    # 标注收缩期
    axes[ch].axvspan(1, 4, alpha=0.1, color='red')

axes[-1].set_xlabel('时间 (秒)', fontsize=12)

plt.tight_layout()
plt.show()

print("✓ 多通道信号绘制完成")

## 6. 统计特性可视化

分析信号的统计特性，如幅度分布、RMS随时间的变化等。

In [ ]:
# 计算滑动窗口RMS
window_size = 100  # 100ms窗口
step = 50  # 50ms步长

rms_values = []
time_points = []

for i in range(0, len(signal_emg) - window_size, step):
    window = signal_emg[i:i+window_size]
    rms = np.sqrt(np.mean(window**2))
    rms_values.append(rms)
    time_points.append(t[i + window_size//2])

# 创建可视化
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# 1. 原始信号
axes[0].plot(t, signal_emg, linewidth=0.5, color='blue', alpha=0.7)
axes[0].set_ylabel('幅度 (mV)', fontsize=12)
axes[0].set_title('原始EMG信号', fontsize=12)
axes[0].grid(True, alpha=0.3)

# 2. RMS随时间变化
axes[1].plot(time_points, rms_values, linewidth=2, color='red')
axes[1].set_ylabel('RMS (mV)', fontsize=12)
axes[1].set_title('RMS随时间变化（反映肌肉激活程度）', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=np.mean(rms_values[:20]), color='green', 
                linestyle='--', label='静息期平均RMS')
axes[1].legend()

# 3. 幅度分布直方图
axes[2].hist(signal_emg, bins=100, alpha=0.7, color='blue', edgecolor='black')
axes[2].set_xlabel('幅度 (mV)', fontsize=12)
axes[2].set_ylabel('频数', fontsize=12)
axes[2].set_title('信号幅度分布（接近正态分布）', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印统计信息
print("\n统计信息:")
print(f"  信号均值: {np.mean(signal_emg):.6f} mV")
print(f"  信号标准差: {np.std(signal_emg):.4f} mV")
print(f"  静息期RMS: {np.mean(rms_values[:20]):.4f} mV")
print(f"  收缩期RMS: {np.max(rms_values):.4f} mV")
print(f"  RMS增益: {np.max(rms_values)/np.mean(rms_values[:20]):.2f}倍")

## 7. 实战：可视化真实数据（可选）

如果你已经有样本数据，可以使用下面的代码进行可视化。

In [ ]:
# 尝试加载真实数据
try:
    from code.week05_data_processing.data_loading import EMGDataLoader
    
    loader = EMGDataLoader(data_dir='../data/sample/', fs=1000)
    
    # 加载一个试验
    time_real, signals_real, label = loader.load_single_trial(
        '../data/sample/subject_01/fist/trial_001.csv'
    )
    
    # 绘制真实数据
    fig, axes = plt.subplots(signals_real.shape[1], 1, figsize=(14, 10))
    fig.suptitle(f'真实EMG数据 - 手势: {label}', fontsize=16, fontweight='bold')
    
    for ch in range(signals_real.shape[1]):
        axes[ch].plot(time_real, signals_real[:, ch], linewidth=0.5)
        axes[ch].set_ylabel(f'通道{ch}\n(mV)', fontsize=10)
        axes[ch].grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('时间 (秒)', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print("✓ 真实数据可视化完成")
    
except FileNotFoundError:
    print("⚠️  未找到样本数据")
    print("请运行: python tools/generate_sample_data.py")
except Exception as e:
    print(f"加载数据时出错: {e}")

## 总结

本教程中，你学习了：

### ✅ 掌握的技能
1. **时域可视化** - 绘制EMG信号的时域波形
2. **频域分析** - 使用功率谱分析频率成分
3. **时频分析** - 使用时频谱图观察频率随时间变化
4. **多通道显示** - 同时可视化多个通道的信号
5. **统计分析** - 计算和可视化信号的统计特性

### 🎯 关键要点
- EMG信号在时域上表现为随机的突发性信号
- 频域分析显示EMG的主要能量集中在20-500 Hz
- 收缩期的RMS值明显高于静息期
- 多通道记录可以捕获更多信息

### ⏭️ 下一步
- 继续学习 [02_滤波器设计实验.ipynb](02_滤波器设计实验.ipynb)
- 学习如何去除噪声和干扰
- 提取有用的特征用于分类